# Code for Large Scale Testing



In [ ]:
!pip install ragatouille

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.1/116.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 837.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Global Variables

In [ ]:
COLLECTION = 'collection_merged.tsv'
PROFILE_COLLECTION = 'profile_collection.tsv'

INDEX_NAME = 'index_merged'
CURRENT_BUCKET = '46-above'
# CURRENT_BUCKET = '36-45'
USER_PROFILE_REC = f'train_{CURRENT_BUCKET}.csv'
USER_TEST_REC = f'test_{CURRENT_BUCKET}.csv'
PROFILE_SIZE = 10
CHUNK_SIZE = 32
ALPHA_DECAY = 0.02
BETA_FREQ = 0.5

# print bucket info
print(f"Current Bucket: {CURRENT_BUCKET}")

Current Bucket: 46-above


## Mount Google Drive

To load files for indexing.

In [ ]:
#Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os

drive_path = "/content/drive/My Drive/full testing"  # Change this path if your files are in a subfolder
files = os.listdir(drive_path)
print(files)


['test_10-15.csv', 'train_10-15.csv', 'test_16-25.csv', 'train_16-25.csv', 'train_26-35.csv', 'test_26-35.csv', 'train_36-45.csv', 'test_36-45.csv', 'train_46-above.csv', 'test_46-above_limited.csv', 'collection_complete.tsv', 'old', 'profile_collection.tsv', 'merged_collection.tsv', 'index_merged_aleena.zip', 'index_complete_aleena.zip', 'index_merged.zip']


In [ ]:
COLLECTION = f'{drive_path}/merged_collection.tsv'
PROFILE_COLLECTION = f'{drive_path}/profile_collection.tsv'
INDEX_NAME = 'index_merged'
USER_PROFILE_REC = f'{drive_path}/train_{CURRENT_BUCKET}.csv'
USER_TEST_REC = f'{drive_path}/test_{CURRENT_BUCKET}.csv'
USER_TEST_REC = f'{drive_path}/test_{CURRENT_BUCKET}_limited.csv'


## Load Collection

Must be in 'tsv' file format where first column is docID/pid and second column is text.

Should be FILTERED to contain only the documents present in Test records.

In [ ]:
import pandas as pd


collection = pd.read_csv(COLLECTION, sep='\t')
print(collection.columns)
collection['content'] = collection['content'].fillna('')


Index(['docID', 'title', 'content'], dtype='object')


In [ ]:
profile_collection = pd.read_csv(PROFILE_COLLECTION, sep='\t')
profile_collection['content'] = profile_collection['content'].fillna('')

print(profile_collection.head())

      docID                                              title  \
0   d-10019                          boeing the boeing company   
1  d-100203       panthers home carolina panthers panthers com   
2   d-10022  medal of honor silver star and distinguished s...   
3   d-10026                                  www ac del co com   
4  d-100360                                   lacrosse america   

                                             content  
0  the boeing company official website safety qua...  
1  panthers home carolina panthers panthers com s...  
2  home of heroes medal of honor military history...  
3  oem aftermarket auto parts genuine gm parts ac...  
4  lacrosse america events events college coaches...  


In [ ]:
print(collection.columns)

Index(['docID', 'title', 'content'], dtype='object')


In [ ]:
# Create a list where each entry contains both 'title' and 'text' with labels
collection_list = ["Title: " + str(title) + " Text: " + str(text)
                   for title, text in zip(collection['title'], collection['content'])]

# Convert docID to a string and store in a list
collection_ids = collection['docID'].astype(str).tolist()

## Creating and Saving Index

### Create Index

Pass text content and docIDs as parameters

In [ ]:
from ragatouille import RAGPretrainedModel
from ragatouille.utils import get_wikipedia_page

RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")
index_path = RAG.index(index_name=INDEX_NAME, collection=collection_list, document_ids=collection_ids)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


artifact.metadata:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/405 [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextManager()
1it [00:04,  4.47s/it]
  0%|          | 0/1 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/colbert/indexing/codecs/residual_embeddings.py:86: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they 

[Feb 15, 12:20:24] #> Optimizing IVF to store map from centroids to list of pids..
[Feb 15, 12:20:24] #> Building the emb2pid mapping..
[Feb 15, 12:20:24] len(emb2pid) = 183518



100%|██████████| 4096/4096 [00:00<00:00, 59320.70it/s]

[Feb 15, 12:20:24] #> Saved optimized IVF to .ragatouille/colbert/indexes/large_index/ivf.pid.pt


Done indexing!


### Saving Index Locally

To allow re-use later without regeneration of index

In [ ]:
import os
from google.colab import files

index_directory = f'.ragatouille/colbert/indexes/{INDEX_NAME}'

zip_filename = f'{INDEX_NAME}.zip'

# Compress the directory into a zip file
!zip -r {zip_filename} {index_directory}

# Download the zip file
files.download(zip_filename)

	zip warning: name not matched: .ragatouille/colbert/indexes/large_index

zip error: Nothing to do! (try: zip -r large_index.zip . -i .ragatouille/colbert/indexes/large_index)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Loading Existing Index

In [ ]:
#Load and unzip saved index

import zipfile

# zip_path = f'/content/{INDEX_NAME}.zip'
zip_path = f'/content/drive/My Drive/full testing/{INDEX_NAME}.zip'

# Directory to extract the files
extract_dir = f'/content/{INDEX_NAME}'

# Extract the ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

In [ ]:
index_path = f'/content/{INDEX_NAME}/.ragatouille/colbert/indexes/{INDEX_NAME}'

In [ ]:
import os
from ragatouille import RAGPretrainedModel

RAG = RAGPretrainedModel.from_index(index_path)

/usr/local/lib/python3.11/dist-packages/colbert/utils/amp.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()


## Load User Profile Records

User Records must already be split into records for profile generation and records for testing.

In [ ]:
import pandas as pd

In [ ]:
user_records = pd.read_csv(USER_PROFILE_REC)

In [ ]:
test_records = pd.read_csv(USER_TEST_REC)

# Generating Profiles with Recency & Freq Info

In [ ]:
from datetime import datetime

def parse_timestamp(t):
    # You can adjust this depending on your time format
    # If your time is in "YYYY-MM-DD" or a Unix epoch etc.
    return pd.to_datetime(t, errors='coerce')

user_records["QueryTime"] = user_records["QueryTime"].apply(parse_timestamp)
test_records["QueryTime"] = test_records["QueryTime"].apply(parse_timestamp)

In [ ]:
# We'll assume "QueryIndex" is the actual query string. If it's an ID, adapt accordingly.
user_query_counts = (
    user_records.groupby(["AnonID", "QueryIndex"]).size().reset_index(name="freq")
)
# This gives a df with columns [AnonID, QueryIndex, freq].
# We'll convert it into a dictionary for quick lookups:
freq_dict = {}
for _, row in user_query_counts.iterrows():
    user, q, f = row["AnonID"], row["QueryIndex"], row["freq"]
    freq_dict.setdefault(user, {})[q] = f


In [ ]:
print(freq_dict)

{10900: {'q-1357': 1, 'q-1402': 1, 'q-1410': 1, 'q-1411': 1, 'q-1415': 14, 'q-1416': 1, 'q-1418': 1, 'q-1419': 1, 'q-1422': 1, 'q-1423': 1, 'q-1425': 1, 'q-1428': 1, 'q-1429': 1, 'q-1430': 1, 'q-1433': 1, 'q-1435': 1, 'q-1438': 1, 'q-1440': 1, 'q-1441': 1, 'q-1444': 1, 'q-1446': 1, 'q-1447': 1, 'q-1449': 1, 'q-1460': 1, 'q-1461': 1, 'q-642': 1, 'q-80': 1}, 40078: {'q-318': 4, 'q-408': 2, 'q-4807': 2, 'q-4987': 1, 'q-4989': 1, 'q-4993': 1, 'q-4994': 6, 'q-5000': 1, 'q-5002': 1, 'q-5009': 1, 'q-5014': 1, 'q-5020': 1, 'q-5034': 1, 'q-5035': 1, 'q-5040': 2, 'q-5042': 2, 'q-5044': 2, 'q-5045': 1, 'q-5049': 1, 'q-5050': 1, 'q-5051': 1, 'q-5053': 1, 'q-5057': 1, 'q-5063': 1, 'q-5064': 1, 'q-5065': 1, 'q-5066': 1}, 62917: {'q-299': 1, 'q-4173': 1, 'q-7245': 1, 'q-8003': 1, 'q-8014': 1, 'q-8017': 2, 'q-8019': 2, 'q-8027': 1, 'q-8036': 1, 'q-8040': 1, 'q-8047': 1, 'q-8049': 1, 'q-8050': 1, 'q-8051': 1, 'q-8052': 1, 'q-8055': 1, 'q-8060': 1, 'q-8061': 1, 'q-8064': 1, 'q-8066': 1, 'q-8073': 1, 'q-

## Creating Profile

In [ ]:
# collection.set_index("docID", inplace=True)
profile_collection.set_index("docID", inplace=True)

### Profile from entire train records

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

def build_user_profiles(train_records, collection, freq_dict):
    """
    Build user profiles so that each (query, clicked_doc) pair is stored once per user.

    :param train_records: DataFrame with columns:
        AnonID, QueryIndex, QueryTime, CandiList, ClickPos, DocsCount
    :param collection: DataFrame or dict indexed by docID with title, text, etc.
    :param freq_dict: dict { user: { query: freq } }, storing frequency of each query for each user
    :return: user_profiles dict structured as:
        {
            user_id: {
                (query, clicked_doc_id): {
                    "query": ...,
                    "clicked_doc_id": ...,
                    "profile_text": ...,
                    "freq": <int or float>,
                    "timestamps": [list_of_datetimes]
                },
                ...
            },
            ...
        }
    """

    user_profiles = {}

    for idx, row in tqdm(train_records.iterrows(), total=len(train_records), desc="Building User Profiles"):
        user = row["AnonID"]
        query = row["QueryIndex"]
        query_time = row["QueryTime"]
        candi_list = [doc.strip("' ") for doc in row['CandiList'].strip('[]').split(',')]
        click_pos = row["ClickPos"]    # 0-based index
        docs_count = row["DocsCount"]

        # If click_pos is valid, identify the clicked doc
        # (Handle edge cases: if click_pos=0 or > len(candi_list), skip, etc.)
        if click_pos >= 0 and click_pos <= len(candi_list):
            clicked_doc_id = candi_list[click_pos]
        else:
            # If no valid click, skip
            continue

        if clicked_doc_id in collection.index:
            # print("Clicked doc id in collection")
            doc_row = collection.loc[clicked_doc_id]
            doc_title = doc_row["title"] if pd.notna(doc_row["title"]) else ""
            doc_text = doc_row["content"] if pd.notna(doc_row["content"]) else ""

        else:
            # doc not found in collection, skip
            continue

        # Frequency
        f = freq_dict.get(user, {}).get(query, 1)  # default freq = 1 if not found

        # Build the profile text
        profile_text = f"[TITLE] {doc_title} [CONTENT] {doc_text}"

        # Insert into user_profiles
        user_profiles.setdefault(user, {})
        key = (query, clicked_doc_id)

        if key not in user_profiles[user]:
            # Create new entry
            user_profiles[user][key] = {
                "query": query,
                "clicked_doc_id": clicked_doc_id,
                "profile_text": profile_text,
                "freq": f,
                "timestamp": query_time if pd.notna(query_time) else None
            }
        else:
            # Update existing entry
            existing_entry = user_profiles[user][key]
            existing_entry["freq"] = max(existing_entry["freq"], f)

            # Append timestamp
            if pd.notna(query_time):
              existing_entry["timestamp"] = max(existing_entry["timestamp"], query_time) if existing_entry["timestamp"] else query_time

    return user_profiles



In [ ]:
user_profiles = build_user_profiles(user_records, profile_collection, freq_dict)

Building User Profiles: 100%|██████████| 3000/3000 [00:00<00:00, 10109.65it/s]


In [ ]:
# print first 5 eneteries of user profiles
print(list(user_profiles.items())[:5])

[(10900, {('q-1402', 'd-8643'): {'query': 'q-1402', 'clicked_doc_id': 'd-8643', 'profile_text': '[TITLE] credit card mortgage banking auto chase online chase com [CONTENT] credit card mortgage banking auto chase online chase com skip to main content please update your browser we don t support this browser version anymore using an updated version will help protect your accounts and provide a better experience update your browser please update your browser we don t support this browser version anymore using an updated version will help protect your accounts and provide a better experience update your browser close hi looking for the uk site visit chase co uk close we ve signed you out of your account you ve successfully signed out we ve enhanced our platform for chase com for a better experience download the chase app for your iphone or android or go to system requirements from your laptop or desktop close this message search search checking savings cds credit cards home loans auto inves

### Profile of a Given Size

In [ ]:
from tqdm import tqdm

def build_user_profiles_limited(train_records, collection, freq_dict, profile_size):
    """
    Build user profiles using only the most recent records up to profile_size.

    :param train_records: DataFrame with columns:
        AnonID, QueryIndex, QueryTime, CandiList, ClickPos, DocsCount
    :param collection: DataFrame or dict indexed by docID with title, text, etc.
    :param freq_dict: dict { user: { query: freq } }, storing frequency of each query for each user
    :param profile_size: Number of records to use per user for profile construction.
    :return: user_profiles dict.
    """

    user_profiles = {}

    # Sort train records by QueryTime (descending) to use the most recent records
    train_records = train_records.sort_values(by=["AnonID", "QueryTime"], ascending=[True, False])

    for user, user_records in tqdm(train_records.groupby("AnonID"), desc="Building User Profiles"):
        user_records = user_records.head(profile_size)  # Select the most recent records

        user_profiles[user] = {}

        for _, row in user_records.iterrows():
            query = row["QueryIndex"]
            query_time = row["QueryTime"]
            candi_list = [doc.strip("' ") for doc in row['CandiList'].strip('[]').split(',')]
            click_pos = row["ClickPos"]

            if click_pos >= 0 and click_pos < len(candi_list):
                clicked_doc_id = candi_list[click_pos]
            else:
                continue

            if clicked_doc_id in collection.index:
                doc_row = collection.loc[clicked_doc_id]
                doc_title = doc_row["title"] if pd.notna(doc_row["title"]) else ""
                doc_text = doc_row["content"] if pd.notna(doc_row["content"]) else ""
            else:
                continue

            # Frequency
            f = freq_dict.get(user, {}).get(query, 1)  # Default frequency = 1 if not found

            # Build the profile text
            profile_text = f"[TITLE] {doc_title} [TEXT] {doc_text}"

            key = (query, clicked_doc_id)

            if key not in user_profiles[user]:
                user_profiles[user][key] = {
                    "query": query,
                    "clicked_doc_id": clicked_doc_id,
                    "profile_text": profile_text,
                    "freq": f,
                    "timestamp": query_time if pd.notna(query_time) else None
                }
            else:
                existing_entry = user_profiles[user][key]
                existing_entry["freq"] = max(existing_entry["freq"], f)
                if pd.notna(query_time):
                    existing_entry["timestamp"] = max(existing_entry["timestamp"], query_time) if existing_entry["timestamp"] else query_time

    return user_profiles

In [ ]:
user_profiles = build_user_profiles_limited(user_records, collection, freq_dict, PROFILE_SIZE)

Building User Profiles: 100%|██████████| 18/18 [00:00<00:00, 916.60it/s]


In [ ]:
print(user_profiles)

{13119: {}, 118401: {}, 126251: {}, 161798: {}, 214011: {}, 244516: {}, 246868: {}, 338759: {}, 416589: {}, 427439: {}, 484492: {}, 509841: {}, 512283: {}, 521651: {}, 632400: {}, 644243: {}, 660007: {}, 687898: {}, 702772: {}, 785557: {}, 853737: {}, 897342: {}, 908625: {}, 1015479: {}, 1068384: {}, 1086026: {}, 1092515: {}, 1154180: {}, 1172510: {}, 1172900: {}, 1193403: {}, 1247541: {}, 1270005: {}, 1326581: {}, 1330676: {}, 1381595: {}, 1509214: {}, 1540418: {}, 1574495: {}, 1585452: {}, 1590685: {}, 1611846: {}, 1644451: {}, 1666956: {}, 1689733: {}, 1849161: {}, 1852124: {}, 1854516: {}, 1969098: {}, 1990514: {}, 2012892: {}, 2029762: {}, 2037120: {}, 2103248: {}, 2112685: {}, 2122680: {}, 2151062: {}, 2172061: {}, 2178436: {}, 2181039: {}, 2181850: {}, 2193560: {}, 2245939: {}, 2251948: {}, 2289105: {}, 2295488: {}, 2302194: {}, 2306700: {}, 2326338: {}, 2341322: {}, 2388758: {}, 2436616: {}, 2525741: {}, 2545771: {}, 2560963: {}, 2587816: {}, 2665580: {}, 2762435: {}, 2823081: 

# Testing

In [ ]:
import pandas as pd
import numpy as np
import json
from tqdm import tqdm

In [ ]:
import nltk
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize

# Function to split a string into chunks of 32 tokens
def chunk_user_profile(profile, chunk_size=16):
    tokens = word_tokenize(profile)  # Tokenize the string
    chunks = [" ".join(tokens[i:i + chunk_size]) for i in range(0, len(tokens), chunk_size)]
    return chunks

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
def compute_weight(delta_days, freq, alpha=ALPHA_DECAY, beta=BETA_FREQ):
    # delta_days can be negative if test query is older? Usually test queries > train queries
    # For safety, take absolute if needed or ensure test > train. We'll assume test_time >= train_time.
    return np.exp(-alpha * delta_days) * np.log(1 + beta * freq)

In [ ]:
def format_results(sorted_results, original_clickpos, clicked_doc):
    """
    Formats results in a structure matching the baseline output.
    """
    print("the inputs are", sorted_results, original_clickpos, clicked_doc)
    new_candi_list, scores = [], []
    for doc in sorted_results:
        new_candi_list.append(doc[0])
        scores.append(doc[1])

    # new_candi_list = [doc['document_id'] for doc in sorted_results]
    # print("type of sorted results",type(sorted_results),"values:", sorted_results)
    # print("type of new_candi_list",type(new_candi_list),"values:", new_candi_list)

    # scores = [doc['score'] for doc in sorted_results]
    new_clickpos = new_candi_list.index(clicked_doc) if clicked_doc in new_candi_list else -1

    return {
        "CandiList": new_candi_list,
        "Scores": scores,
        "OriginalClickPos": original_clickpos,
        "NewClickPos": new_clickpos
    }

In [ ]:
print(collection.columns)
# print collection index top 5
# print(collection.head())

Index(['docID', 'title', 'content'], dtype='object')


In [ ]:
# index is the collection id

# # collection_ids = collection['docID'].astype(str).tolist()
# collection_ids = collection.index.astype(str).tolist()
collection.set_index("docID", inplace=True)


valid_doc_ids = set(collection.index)
print(len(valid_doc_ids),len(collection_ids))


4858 4858


In [ ]:
def rerank_user_test_queries(user_id, test_query_records, user_profiles, valid_doc_ids, alpha=ALPHA_DECAY, beta=BETA_FREQ):
    """
    test_query_records: subset of test_records for this user
    user_profiles: the dictionary built from train (user_profiles[user_id])
    Returns a list of dict with re-ranked results and metrics.
    """
    results = []

    if user_id not in user_profiles:
        # No profile data => no personalization
        print("No profile data for user", user_id)
        user_profiles[user_id] = {}

    profile_entries = user_profiles[user_id]
    # print("profile_entries: ", profile_entries,"for ", user_id)
    for _, row in test_query_records.iterrows():
        query = row["QueryIndex"]
        query_time = row["QueryTime"]
        raw_candi_list = [doc.strip("' ") for doc in row['CandiList'].strip('[]').split(',')]
        # print("raw_candi_list: ", raw_candi_list)
        candi_list = [doc for doc in raw_candi_list if doc in valid_doc_ids]
        # print("the length of candi_list is:", len(candi_list), "values:", candi_list)
        if not candi_list:
          continue
        # click_pos = row["ClickPos"]  # For metric calculation
        # click pos is the index where row["DocIndex"] is present
        # new correct logic for clicked pos
        click_pos = candi_list.index(row["DocIndex"]) if row["DocIndex"] in raw_candi_list else -1
        # print("click_pos is:", click_pos, candi_list, row["DocIndex"], "profile enteries", profile_entries)

        docs_count = row["DocsCount"]

        final_scores = {}

        # 1) For each (q_i, doc_i) in user profile
        for entry, query_data in profile_entries.items():
            if query_time and query_data["timestamp"] is not None:
                delta_days = (query_time - query_data["timestamp"]).days
            else:
                # If missing times, skip or treat as 0
                delta_days = 0

            # compute weight
            w_i = compute_weight(delta_days, query_data["freq"], alpha, beta)
            if w_i <= 0:
                continue  # negligible weight => skip

            # chunk and rerank
            chunks = chunk_user_profile(query_data["profile_text"], CHUNK_SIZE)

            # gather chunk results
            chunk_scores = {}
            for ch in chunks:
                rag_results = RAG.search(ch, doc_ids=candi_list)  # Replace with RAG.search(ch, ...)
                # print("Lenght of rag_results: ", rag_results)
                for res in rag_results:
                    doc_id = res["document_id"]
                    score = res["score"]
                    chunk_scores[doc_id] = max(chunk_scores.get(doc_id, 0), score)

            # now multiply chunk_scores by w_i and add to final_scores
            for doc_id, sc in chunk_scores.items():
                final_scores[doc_id] = final_scores.get(doc_id, 0.0) + w_i * sc

        # print("final_scores: ", final_scores)

        # 2) sort final_scores
        ranked_list = sorted(final_scores.items(), key=lambda x: x[1], reverse=True)

        clicked_doc_id = candi_list[click_pos] if click_pos >= 0 and click_pos < len(candi_list) else None
        formatted_results = format_results(ranked_list, click_pos, clicked_doc_id)

        # Store structured results in a list format (same as baseline)
        results.append({
            "user_id": user_id,
            "query": query,
            **formatted_results
        })

    return results


In [ ]:
def run_experiment(train_records, test_records, user_profiles, valid_doc_ids):
    # We gather results in a list, then compute metrics
    all_results = []

    # Let's group test_records by user
    test_grouped = test_records.groupby("AnonID")

    count=0
    for user_id, group_df in tqdm(test_grouped, desc="Re-ranking Test Queries"):
        print("user_id: ", user_id)
        user_results = rerank_user_test_queries(user_id, group_df, user_profiles, valid_doc_ids)
        all_results.extend(user_results)

    return all_results


In [ ]:
results = run_experiment(user_records, test_records, user_profiles, valid_doc_ids)

Re-ranking Test Queries:   0%|          | 0/75 [00:00<?, ?it/s]

user_id:  10900
Loading searcher for index index_merged for the first time... This may take a few seconds
[Feb 21, 15:13:00] #> Loading codec...
[Feb 21, 15:13:00] #> Loading IVF...
[Feb 21, 15:13:00] #> Loading doclens...


/usr/local/lib/python3.11/dist-packages/colbert/utils/amp.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()
/usr/local/lib/python3.11/dist-packages/colbert/indexing/codecs/residual.py:141: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you s

[Feb 21, 15:13:00] #> Loading codes and residuals...




  0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/colbert/indexing/codecs/residual_embeddings.py:86: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.

...

the inputs are [('d-243', 122.2530226713634), ('d-341290', 119.2904482601454), ('d-878210', 116.77094322312465), ('d-589325', 111.725861332667), ('d-698930', 75.56099598147829)] 2 d-243
user_id:  16497479


/usr/local/lib/python3.11/dist-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextManager()


the inputs are [('d-323526', 192.97522062755468), ('d-622836', 192.97522062755468), ('d-144314', 192.97522062755468), ('d-2028', 192.97522062755468), ('d-401759', 192.97522062755468), ('d-557711', 186.89262254178982)] 3 d-2028
the inputs are [('d-345797', 211.02186646785654), ('d-535088', 210.50779707102654), ('d-446879', 207.77773461331196), ('d-297', 207.4711385851018), ('d-38270', 204.9413313092802)] 2 d-297
the inputs are [('d-100203', 201.98029789191003), ('d-797381', 200.22844255914455), ('d-540383', 195.72791664024032), ('d-927909', 192.82415503834847), ('d-660928', 129.3191650347042), ('d-138308', 119.13957264628726)] 1 d-100203
the inputs are [('d-13634', 177.79613227175352), ('d-1766', 177.79613227175352), ('d-521840', 177.79613227175352), ('d-589582', 177.79613227175352), ('d-580274', 176.635479825332)] 1 d-1766
the inputs are [('d-153128', 174.78050379711968), ('d-31831', 174.78050379711968), ('d-927908', 174.78050379711968), ('d-846586', 165.00089813537323), ('d-317697', 1

Re-ranking Test Queries: 100%|██████████| 75/75 [3:18:22<00:00, 158.70s/it]

the inputs are [('d-1323', 82.3512041973878), ('d-45913', 81.81264761053725), ('d-103943', 79.1139559211982), ('d-79274', 55.78948403614282), ('d-248015', 55.78948403614282), ('d-248016', 55.78948403614282)] 1 d-1323


# Evaluation Metrics

**Mean Reciprocal Rank**

In [ ]:
import pandas as pd

def calculate_mrr(df_results):
    """
    Calculates the Mean Reciprocal Rank (MRR) for a given DataFrame.
    Supports both global and per-user MRR calculations.

    :param df_results: DataFrame with query-level reranking results
    :return: (global_mrr, per_user_mrrs)
    """
    df_valid = df_results[df_results["NewClickPos"] >= 0]  # Ignore cases where click was not found
    df_valid["reciprocal_rank"] = 1 / (df_valid["NewClickPos"] + 1)  # Convert to 1-based rank

    # Compute Global MRR
    global_mrr = df_valid["reciprocal_rank"].mean() if not df_valid.empty else 0.0

    # Compute Per-User MRR
    per_user_mrrs = df_valid.groupby("user_id")["reciprocal_rank"].mean().to_dict()

    return global_mrr, per_user_mrrs


In [ ]:
results_df = pd.DataFrame(results)


global_mrr, per_user_mrr = calculate_mrr(results_df)

print("Global MRR: ", global_mrr)
print("Per User MRR:\n", per_user_mrr)

Global MRR:  0.5621529723000311
Per User MRR:
 {10900: 0.4333333333333333, 40078: 0.5916666666666666, 62917: 0.41666666666666663, 83471: 0.5583333333333333, 123703: 0.5666666666666667, 141233: 0.47222222222222215, 322241: 0.6952380952380952, 443567: 0.5666666666666667, 657778: 0.4107142857142857, 702108: 0.65, 711391: 0.5866666666666667, 756370: 0.4066666666666666, 856648: 0.3527777777777778, 915996: 0.7083333333333333, 961732: 0.5033333333333333, 1080144: 0.4, 1093860: 0.5086805555555556, 1194962: 0.5166666666666667, 1306380: 0.4055555555555556, 1424541: 0.6708333333333334, 1485775: 0.5, 1486231: 0.25, 1490552: 0.6761904761904762, 1517755: 0.3103174603174603, 1659408: 0.9375, 1791126: 0.38095238095238093, 1817741: 0.4797619047619048, 1834478: 0.5694444444444444, 1970261: 0.36666666666666664, 2020847: 0.8055555555555555, 2044086: 0.46875, 2061454: 0.728125, 2102894: 0.38571428571428573, 2104252: 0.39999999999999997, 2114800: 0.5476190476190476, 2199573: 0.6875, 2204916: 0.74, 2312684: 

In [ ]:
print(results[0:5])

# store results to dir
with open('results.json', 'w') as f:
    json.dump(results, f)

[{'user_id': 10900, 'query': 'q-1415', 'CandiList': ['d-255974', 'd-521350', 'd-686609', 'd-8776', 'd-88879', 'd-8643', 'd-395332', 'd-29358', 'd-383343', 'd-2694'], 'Scores': [152.02775212150996, 152.02775212150996, 152.02775212150996, 152.02775212150996, 152.02775212150996, 152.02775212150996, 152.02775212150996, 152.02775212150996, 152.02775212150996, 152.02775212150996], 'OriginalClickPos': 5, 'NewClickPos': 5}, {'user_id': 10900, 'query': 'q-1415', 'CandiList': ['d-255974', 'd-521350', 'd-686609', 'd-8776', 'd-88879', 'd-8643', 'd-395332', 'd-29358', 'd-383343', 'd-2694'], 'Scores': [151.0508365965945, 151.0508365965945, 151.0508365965945, 151.0508365965945, 151.0508365965945, 151.0508365965945, 151.0508365965945, 151.0508365965945, 151.0508365965945, 151.0508365965945], 'OriginalClickPos': 5, 'NewClickPos': 5}, {'user_id': 10900, 'query': 'q-1461', 'CandiList': ['d-277205', 'd-124976', 'd-849449', 'd-8726', 'd-879595', 'd-32549', 'd-310589'], 'Scores': [136.19817316068077, 136.19

**Mean Average Precision @ 1**

In [ ]:
import pandas as pd

def calculate_map_at_1(df_results):
    """
    Computes Mean Average Precision at Rank 1 (MAP@1).

    :param df_results: DataFrame containing 'user_id', 'query', 'original_click_pos', 'new_click_pos'
    :return: (global_map1, per_user_map1)
    """
    df_results["precision_at_1"] = (df_results["NewClickPos"] == 0).astype(int)

    # Compute Global MAP@1
    global_map1 = df_results["precision_at_1"].mean()

    # Compute Per-User MAP@1
    per_user_map1 = df_results.groupby("user_id")["precision_at_1"].mean().to_dict()

    return global_map1, per_user_map1


In [ ]:
global_map1, per_user_map1 = calculate_map_at_1(results_df)

print("Global map1: ", global_map1)
print("Per User map1:\n", per_user_map1)

Global map1:  0.3411764705882353
Per User map1:
 {10900: 0.2857142857142857, 40078: 0.4, 62917: 0.2, 83471: 0.3, 123703: 0.3333333333333333, 141233: 0.3333333333333333, 322241: 0.6, 443567: 0.4, 657778: 0.14285714285714285, 702108: 0.4, 711391: 0.3, 756370: 0.2, 856648: 0.0, 915996: 0.5, 961732: 0.2, 1080144: 0.0, 1093860: 0.375, 1194962: 0.3333333333333333, 1306380: 0.16666666666666666, 1424541: 0.5, 1485775: 0.2, 1486231: 0.0, 1490552: 0.5714285714285714, 1517755: 0.0, 1659408: 0.875, 1791126: 0.14285714285714285, 1817741: 0.2857142857142857, 1834478: 0.3333333333333333, 1970261: 0.0, 2020847: 0.6666666666666666, 2044086: 0.125, 2061454: 0.625, 2102894: 0.14285714285714285, 2104252: 0.16666666666666666, 2114800: 0.14285714285714285, 2199573: 0.5, 2204916: 0.6, 2312684: 0.16666666666666666, 2332453: 0.0, 2402080: 0.2, 2425308: 0.2, 2439668: 0.0, 2528943: 0.75, 2560293: 0.6, 2651358: 1.0, 2752882: 0.8, 2941252: 0.0, 2992908: 0.125, 2995358: 0.4, 3014747: 0.625, 3041214: 0.2, 3057367: 0

**Combine MRR and MAP into a single file**

In [ ]:
import pandas as pd

def combine_user_metrics(user_mrrs, user_map1):
    """
    Combines per-user MRR, MAP@1, and click position change metrics into a single DataFrame.
    Ensures consistent data types for merging.

    :param user_mrrs: Dictionary {user_id (int/str) -> MRR}
    :param user_map1: Dictionary {user_id (int/str) -> MAP@1}
    :return: DataFrame with combined user-level metrics.
    """

    # Convert user IDs to strings for consistent merging
    user_mrrs = {str(user): mrr for user, mrr in user_mrrs.items()}
    user_map1 = {str(user): map1 for user, map1 in user_map1.items()}
    # Convert dictionaries to DataFrames
    df_mrr = pd.DataFrame(user_mrrs.items(), columns=["user_id", "MRR"])
    df_map1 = pd.DataFrame(user_map1.items(), columns=["user_id", "MAP@1"])

    # Convert all user_id columns to string before merging
    df_mrr["user_id"] = df_mrr["user_id"].astype(str)
    df_map1["user_id"] = df_map1["user_id"].astype(str)
    # Merge all DataFrames on user_id
    df_combined = df_mrr.merge(df_map1, on="user_id", how="outer")

    # Save to CSV & JSONL
    df_combined.to_csv("per_user_metrics.csv", index=False)
    df_combined.to_json("per_user_metrics.jsonl", orient="records", lines=True)

    print("✅ Per-user metrics saved: per_user_metrics.csv & per_user_metrics.jsonl")

    return df_combined

In [ ]:
combined_metrics = combine_user_metrics(per_user_mrr, per_user_map1)

✅ Per-user metrics saved: per_user_metrics.csv & per_user_metrics.jsonl


**Click Position Changes**

In [ ]:
def evaluate_click_pos_changes(df_results):
    """
    Evaluates changes in click positions and computes improvement metrics.

    :param df_results: DataFrame with query-level reranking results
    :return: Dictionary with click position change metrics
    """
    total_queries = len(df_results)

    # Categorize click position changes
    df_results["click_change"] = df_results["NewClickPos"] - df_results["OriginalClickPos"]
    improved = (df_results["click_change"] < 0).sum()
    worsened = (df_results["click_change"] > 0).sum()
    unchanged = (df_results["click_change"] == 0).sum()
    not_found = (df_results["NewClickPos"] == -1).sum()

    # Per-User Click Change Stats
    user_click_pos = df_results.groupby("user_id")["click_change"].apply(lambda x: {
        "avg_change": x.mean(),
        "num_improved": (x < 0).sum(),
        "num_worsened": (x > 0).sum(),
        "num_unchanged": (x == 0).sum(),
        "num_not_found": (x == -1).sum(),
    }).to_dict()

    # Compute percentages
    improved_percent = (improved / total_queries) * 100 if total_queries > 0 else 0
    worsened_percent = (worsened / total_queries) * 100 if total_queries > 0 else 0
    unchanged_percent = (unchanged / total_queries) * 100 if total_queries > 0 else 0
    not_found_percent = (not_found / total_queries) * 100 if total_queries > 0 else 0

    return {
        "total_queries": total_queries,
        "improved": improved,
        "improved_percent": improved_percent,
        "worsened": worsened,
        "worsened_percent": worsened_percent,
        "unchanged": unchanged,
        "unchanged_percent": unchanged_percent,
        "not_found": not_found,
        "not_found_percent": not_found_percent,
        "user_click_pos": user_click_pos
    }

In [ ]:
click_pos_changes = evaluate_click_pos_changes(results_df)

print(click_pos_changes)

{'total_queries': 510, 'improved': 228, 'improved_percent': 44.70588235294118, 'worsened': 144, 'worsened_percent': 28.235294117647058, 'unchanged': 138, 'unchanged_percent': 27.058823529411764, 'not_found': 0, 'not_found_percent': 0.0, 'user_click_pos': {(10900, 'avg_change'): -0.14285714285714285, (10900, 'num_improved'): 2.0, (10900, 'num_worsened'): 2.0, (10900, 'num_unchanged'): 3.0, (10900, 'num_not_found'): 0.0, (40078, 'avg_change'): -0.2, (40078, 'num_improved'): 3.0, (40078, 'num_worsened'): 3.0, (40078, 'num_unchanged'): 4.0, (40078, 'num_not_found'): 1.0, (62917, 'avg_change'): 1.4, (62917, 'num_improved'): 0.0, (62917, 'num_worsened'): 4.0, (62917, 'num_unchanged'): 1.0, (62917, 'num_not_found'): 0.0, (83471, 'avg_change'): -1.2, (83471, 'num_improved'): 7.0, (83471, 'num_worsened'): 0.0, (83471, 'num_unchanged'): 3.0, (83471, 'num_not_found'): 3.0, (123703, 'avg_change'): -0.5, (123703, 'num_improved'): 3.0, (123703, 'num_worsened'): 3.0, (123703, 'num_unchanged'): 0.0, (

# Save Results

In [ ]:
def save_results(results, filename):
    results_list = []

    for entry in results:
        results_list.append({
            "user_id": entry["user_id"],
            "query": entry["query"],
            "new_candi_list": entry["CandiList"],
            "scores":entry["Scores"],
            "original_click_pos": entry["OriginalClickPos"],
            "new_click_pos": entry["NewClickPos"],
            "click_change": entry["NewClickPos"] - entry["OriginalClickPos"]
        })

    df_results = pd.DataFrame(results_list)

    # Save to CSV and JSONL
    df_results.to_csv(f"{filename}.csv", index=False)
    df_results.to_json(f"{filename}.jsonl", orient="records", lines=True)

    print(f"Results saved: {filename}.csv & {filename}.jsonl")

    return df_results

In [ ]:
saved_results = save_results(results, 'recency_freq_60_5')

Results saved: recency_freq_60_5.csv & recency_freq_60_5.jsonl


In [ ]:
# prompt: save all the files in the root directory that are not in any other folder to a new results folder on google drive

import os
import shutil

# Define source and destination paths
source_dir = "/content"
destination_dir = "/content/drive/My Drive/rafay_results_2/"+str(CURRENT_BUCKET)+"/"

# Create the destination directory if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

# Iterate through files in the source directory
for filename in os.listdir(source_dir):
    source_path = os.path.join(source_dir, filename)

    # Check if it's a file (not a directory) and not already in a subdirectory
    if os.path.isfile(source_path) and not any(part.startswith(".") for part in filename.split("/")):
        destination_path = os.path.join(destination_dir, filename)

        # Copy the file
        shutil.copy2(source_path, destination_path)  # copy2 preserves metadata
        print(f"Copied '{filename}' to '{destination_dir}'")

Copied 'recency_freq_60_5.jsonl' to '/content/drive/My Drive/rafay_results_2/46-above/'
Copied 'results.json' to '/content/drive/My Drive/rafay_results_2/46-above/'
Copied 'per_user_metrics.csv' to '/content/drive/My Drive/rafay_results_2/46-above/'
Copied 'per_user_metrics.jsonl' to '/content/drive/My Drive/rafay_results_2/46-above/'
Copied 'recency_freq_60_5.csv' to '/content/drive/My Drive/rafay_results_2/46-above/'
